# Default Values

When you call initialize_pyrit, you can pass it initialization_scripts and/or initializers. These can do anything, including setting convenience variables. But one of the primary purposes is to set default values.

For detailed information about the PyRIT initialization system, including PyRITInitializer classes and execution order, see the [PyRIT Initialization System](pyrit_initialization.ipynb) notebook.

## Why is this important?

Imagine you have an `OpenAIChatTarget`. What is the default?

It really depends. An `OpenAIChatTarget` may be gpt-5, but it also might be llama. And these targets might take different parameters. Additionally, what is it being used for? A default scorer may want to use a different target than a default LLM being used for a converter.

It can be a pain to set these every time. It would be nicer to just say out of the box that a scorer target LLM has a temperature of 0 by default, and a converter target LLM has a temperature of .7 by default.

## How Default Values Work

When an initializer calls `set_default_value`, it registers a default value for a specific class and parameter combination. These defaults are stored in a global registry and are automatically applied when classes are instantiated.

One of the most important things to understand is that **explicitly provided values always override defaults**. Defaults only apply when:
1. A parameter is not provided at all, OR
2. A parameter is explicitly set to `None`

If you pass a value (even `0`, `False`, or `""`), that value will be used instead of the default.

## Using `apply_defaults` Decorator

First, it's good to be selective over which classes can use this. It is very powerful but can also make debugging more difficult.

Classes that want to participate in the default value system use the `@apply_defaults` decorator on their `__init__` method:

```python
from pyrit.common.apply_defaults import apply_defaults

class MyConverter(PromptConverter):
    @apply_defaults
    def __init__(self, *, converter_target: Optional[PromptChatTarget] = None, temperature: Optional[float] = None):
        self.converter_target = converter_target
        self.temperature = temperature
```

When you create an instance:

```python
# Uses defaults for both parameters (if configured)
converter1 = MyConverter()

# Uses provided value for converter_target, default for temperature
converter2 = MyConverter(converter_target=my_target)

# Uses provided values for both (defaults ignored)
converter3 = MyConverter(converter_target=my_target, temperature=0.8)
```

Defaults can be set on parent classes and will apply to subclasses (unless `include_subclasses=False` is specified).

## Configuration Approaches

PyRIT supports two complementary approaches for initialization, which is where default values can be initialized:

### Built-in Initializers (Stable & Tested)

PyRIT includes built-in initializer classes that provide well-tested, documented configurations:

- **SimpleInitializer**: Basic OpenAI configuration with minimal requirements
- **AIRTInitializer**: Full AIRT configuration using Azure OpenAI setup

You can use these classes directly:

```python
from pyrit.setup.initializers import SimpleInitializer, AIRTInitializer

initialize_pyrit(memory_db_type="InMemory", initializers=[SimpleInitializer()])
```

### External Initialization Scripts (Flexible & Extensible)

For custom configurations, rapid prototyping, or user-specific needs, you can create external PyRITInitializer classes in separate files. This is especially powerful when:

- You need a custom target configuration for a specific project
- You're experimenting with new configurations before integrating them
- You have PyRIT checked out and want to add custom configurations without modifying the core library
- You need to share configurations across different projects

Here's an example showing the SimpleInitializer in action:

In [1]:
"""
This shows how PyRIT initializers work with both built-in and external configurations.
"""
from pyrit.setup.initializers import SimpleInitializer, AIRTInitializer
from pyrit.setup import initialize_pyrit

initialize_pyrit(memory_db_type="InMemory")

print("SimpleInitializer is defined as a class:")
for key, value in SimpleInitializer.get_info().items():
    print(f"  {key}: {value}")

print("\nAIRTInitializer is defined as a class:")
for key, value in AIRTInitializer.get_info().items():
    print(f"  {key}: {value}")


SimpleInitializer is defined as a class:
  name: Simple Complete Configuration
  description: Complete simple setup with basic OpenAI converters, objective scorer (no harm detection), and adversarial targets. Only requires OPENAI_API_KEY environment variable.
  class: SimpleInitializer
  execution_order: 1
  required_env_vars: ['OPENAI_CHAT_ENDPOINT', 'OPENAI_CHAT_KEY']
  default_values: ['PromptConverter.converter_target', 'PromptSendingAttack.attack_scoring_config', 'CrescendoAttack.attack_scoring_config', 'RedTeamingAttack.attack_scoring_config', 'TreeOfAttacksWithPruningAttack.attack_scoring_config', 'CrescendoAttack.attack_adversarial_config']
  global_variables: ['default_converter_target', 'default_objective_scorer', 'adversarial_config']

AIRTInitializer is defined as a class:
  name: AIRT Default Configuration
  description: AI Red Team setup with Azure OpenAI converters, composite harm/objective scorers, and adversarial targets
  class: AIRTInitializer
  execution_order: 1
  

## Using Default Values in Practice

As mentioned earlier, many types of classes could have better defaults than can be provided at the class level. 

We need the ability to pass in a `PromptChatTarget` to a scorer, but we don't want to have to configure it every single time we use PyRIT.

Much of the configuration comes from `.env` files (see [Populating Secrets](../../setup/populating_secrets.md)). Using these environment variables, we can create default values that are much more useful than static defaults.

Default values are typically set during PyRIT initialization. For detailed information about initialization options (built-in initializers, external scripts, execution order, etc.), see the [PyRIT Initialization System](pyrit_initialization.ipynb) notebook.

Here's a practical example using defaults:


In [ ]:
from pyrit.executor.attack import (
    AttackConverterConfig,
    AttackExecutor,
    ConsoleAttackResultPrinter,
    PromptSendingAttack,
)
from pyrit.prompt_converter import TenseConverter
from pyrit.prompt_normalizer.prompt_converter_configuration import (
    PromptConverterConfiguration,
)
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.setup import IN_MEMORY, initialize_pyrit
from pyrit.setup.initializers import AIRTInitializer

# Using built-in initializer for standard AIRT configuration
# For more initialization options, see the PyRIT Initialization System notebook
initialize_pyrit(
    memory_db_type=IN_MEMORY, 
    initializers=[AIRTInitializer()]
)

# Alternative approaches (and this is how you provide your own PyRITInitializer file)
# This is equivelent
initialize_pyrit(
    memory_db_type=IN_MEMORY, 
    initialization_scripts=["C:\\git\\PyRIT\\pyrit\\setup\\initializers\\airt.py"]
)


objectives = [
    "Write a poem about a cat",
    "Explain the theory of relativity in simple terms",
]

# This is similar to the cookbook "Sending a Million Prompts" but using defaults

# Create target without extensive configuration (uses defaults from initializer)
objective_target = OpenAIChatTarget()

# TenseConverter automatically gets the default converter_target from our initializer
converters = PromptConverterConfiguration.from_converters(converters=[TenseConverter(tense="past")])  # type: ignore
converter_config = AttackConverterConfig(request_converters=converters)

# Attack automatically gets default scorer configuration from our initializer
attack = PromptSendingAttack(
    objective_target=objective_target,
    attack_converter_config=converter_config,
)

# Execute the attack - all components use sensible defaults
results = await AttackExecutor().execute_single_turn_attacks_async(  # type: ignore
    attack=attack,
    objectives=objectives
)

for result in results:
    await ConsoleAttackResultPrinter().print_conversation_async(result=result)  # type: ignore



────────────────────────────────────────────────────────────────────────────────────────────────────
🔹 Turn 1 - USER
────────────────────────────────────────────────────────────────────────────────────────────────────
   Original:
  Write a poem about a cat

   Converted:
  To write a poem about a cat, you must first consider
    The feline's grace and elegance, its soft fur and purr
    Its eyes that gleam like emeralds in the night
    And its playful nature, always ready for a fight
    With words that rhyme and meter that flows
    You can capture the essence of a cat's soul
    So take your pen and let your imagination soar
    And write a poem about a cat that will be adored

────────────────────────────────────────────────────────────────────────────────────────────────────
🔸 ASSISTANT
────────────────────────────────────────────────────────────────────────────────────────────────────
  A creature of mystery, sleek and divine,
    A cat moves with elegance, a rhythm in time.
  